# Import packages

In [ ]:
import numpy as np
import pandas as pd
import pickle, time
import os
from collections import OrderedDict as odict
from functools import reduce
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix,cohen_kappa_score
from bokeh.plotting import figure,output_file,output_notebook,show
import bokeh

# Implementations of SimHash, FlyHash, and DenseFly

The implementation of these LSH algorithm are adopted from https://github.com/dataplayer12/Fly-LSH

Our code follows MIT License

MIT License

Copyright (c) 2017 Jaiyam Sharma

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# SimHash implementation

In [ ]:
class LSH(object):
    def __init__(self,data,label,hash_length,nnn):
        """
        data: Nxd matrix
        label: N*1 vector
        hash_length: the projection numbers
        nnn: n nearest Neighbors
        """
        self.hash_length=hash_length
        self.data=data-np.mean(data,axis=1,keepdims=True) #preprocessing
        self.label=label
        self.group_counts=np.unique(self.label).shape[0] #get group numbers
        self.weights=np.random.random((data.shape[1],hash_length))#save projection directions
        self.hashes=(self.data@self.weights)>0  #get hash
        self.maxl1distance=2*self.hash_length
        self.nnn=nnn
        
        self.create_bins()
    
    #query one cell in the reference data and retrun the nnn Nearest Neighbors
    def query(self,qidx,nnn,not_olap=False):
        L1_distances=np.sum(np.abs(self.hashes[qidx,:]^self.hashes),axis=1)
        nnn=min(self.hashes.shape[0],nnn)#make nnn reansonable(not too big)
        if not_olap:
            no_overlaps=np.sum(L1_distances==self.maxl1distance)
            return no_overlaps
        NNs=L1_distances.argsort()
        NNs=NNs[(NNs != qidx)][:nnn]
        return NNs
    
    #creat bins for multiprobe
    def create_bins(self):
        if hasattr(self,'bins'):
            return 
        self.bins=np.unique(self.hashes,axis=0)
        self.num_bins=self.bins.shape[0]
        assignment=np.zeros(self.hashes.shape[0])
        for idx,_bin in enumerate(self.bins):
            assignment[(self.hashes==_bin).all(axis=1)]=idx
        self.binstopoints={bin_idx:np.flatnonzero(assignment==bin_idx) for bin_idx in range(self.bins.shape[0])}
        self.pointstobins={point:int(_bin) for point,_bin in enumerate(assignment)}
    
    #multiprobe query, query one cell in the reference dataset and return nnn NNs in the search_radius (if there are)
    def query_bins(self,qidx,search_radius=1,order=True):
        if not hasattr(self,'bins'):
            raise ValueError('Bins for model not created')
        query_bin=self.bins[self.pointstobins[qidx]]
        valid_bins=np.flatnonzero((query_bin[None,:]^self.bins).sum(axis=1)<=search_radius)
        all_points=reduce(np.union1d,np.array([self.binstopoints[idx] for idx in valid_bins]))
        if order:
            l1distances=(self.hashes[qidx,:]^self.hashes[all_points,:]).sum(axis=1)
            all_points=all_points[l1distances.argsort()]
        all_points=all_points[1:self.nnn+1]
        return all_points
    
    # query each cell in the indices and return the confusion matrix
    def get_confusion_matrix(self,indices):
        cm = np.zeros((self.group_counts,self.group_counts))
        for idx in indices:
            pre_label = self.label[self.query(idx,self.nnn)]
            true_label = np.array([self.label[idx]]*pre_label.shape[0])
            cm += confusion_matrix(true_label,pre_label,np.unique(self.label))
        return cm
    
    # compute conhen kappa score of a confusion matrix
    def compute_CKS(self,matrix):
        po = matrix.trace()/np.sum(matrix)
        pe = sum(np.sum(matrix,axis=0)*np.sum(matrix,axis=1))/np.sum(matrix)/np.sum(matrix)
        return (po-pe)/(1-pe)
    
    # five-fold cross validation
    def test_performance(self):
        if not hasattr(self,'bins'):
            self.create_bins()
        CKS = 0
        for step in range(5):
            query_indices = np.random.choice(self.data.shape[0],self.data.shape[0]//5)
            CM = self.get_confusion_matrix(query_indices)
            CKS += self.compute_CKS(CM)/5
        return CKS    

# FlyHash implementation

In [ ]:
class flylsh(LSH):
    def __init__(self,data,label,hash_length,nnn,sampling_ratio,embedding_size):
        """
        data: Nxd matrix
        label: N*1 vector
        hash_length: scalar
        nnn: n nearest neighbors
        sampling_ratio: fraction of input dims to sample from when producing a hash
        embedding_size: dimensionality of projection space, m
        Note that in Flylsh, the hash length and embedding_size are NOT the same
        whereas in usual LSH they are
        """
        self.hash_length=hash_length
        self.embedding_size=embedding_size
        K=embedding_size//hash_length
        self.data=data-np.mean(data,axis=1,keepdims=True) #preprocessing
        self.label=label
        self.group_counts=np.unique(self.label).shape[0]
        self.nnn=nnn
        
        # get high dimension hashes
        num_projections=int(sampling_ratio*data.shape[1])
        weights=np.random.random((data.shape[1],embedding_size))
        yindices=np.arange(weights.shape[1])[None,:]
        xindices=weights.argsort(axis=0)[-num_projections:,:]
        self.weights=np.zeros_like(weights,dtype=np.bool)
        self.weights[xindices,yindices]= True#sparse projection vectors
        
        all_activations=(self.data@self.weights)
        xindices=np.arange(data.shape[0])[:,None]
        yindices=all_activations.argsort(axis=1)[:,-hash_length:]
        self.hashes=np.zeros_like(all_activations,dtype=np.bool)
        self.hashes[xindices,yindices]=True #choose topk activations
        self.dense_activations=all_activations
        self.sparse_activations=self.hashes.astype(np.float32)*all_activations #elementwise product
        self.maxl1distance=2*self.hash_length
        self.lowd_hashes=all_activations.reshape((-1,hash_length,K)).sum(axis=-1) > 0 # get low dimension hashes
        
        self.create_lowd_bins()
    
    #unused 
    def create_highd_bins(self,d,rounds=1):
        """
        This function implements a relaxed binning for FlyLSH
        This is only one of the many possible implementations for such a scheme
        d: the number of bits to match between hashes for putting them in the same bin
        """
        self.highd_bins=self.hashes[0:1,:] #initialize hashes to first point
        self.highd_binstopoints,self.highd_pointstobins={},{i:[] for i in range(self.hashes.shape[0])}
        for round in range(rounds):
            for hash_idx,this_hash in enumerate(self.hashes):
                overlap=(self.maxl1distance-((this_hash[None,:]^self.highd_bins).sum(axis=1)))>=2*d
                #print(overlap.shape)
                if overlap.any():
                    indices=np.flatnonzero(overlap)
                    #indices=indices.tolist()
                    #print(indices)
                    self.highd_pointstobins[hash_idx].extend(indices)
                    for idx in indices:
                        if idx not in self.highd_binstopoints:
                            #print(indices,idx)
                            self.highd_binstopoints[idx]=[]
                        self.highd_binstopoints[idx].append(hash_idx)
                else:
                    self.highd_bins=np.append(self.highd_bins,this_hash[None,:],axis=0)
                    bin_idx=self.highd_bins.shape[0]-1
                    self.highd_pointstobins[hash_idx].append(bin_idx)
                    self.highd_binstopoints[bin_idx]=[hash_idx]
    
    # create bins for multiprobe 
    def create_lowd_bins(self):
        start=time.time()
        self.lowd_bins=np.unique(self.lowd_hashes,axis=0)
        #self.num_bins=self.bins.shape[0]

        assignment=np.zeros(self.lowd_hashes.shape[0])
        for idx,_bin in enumerate(self.lowd_bins):
            assignment[(self.lowd_hashes==_bin).all(axis=1)]=idx
        self.lowd_binstopoints={bin_idx:np.flatnonzero(assignment==bin_idx) for bin_idx in range(self.lowd_bins.shape[0])}
        self.lowd_pointstobins={point:int(_bin) for point,_bin in enumerate(assignment)}
        self.timetoindex=time.time()-start
    
    # multiprobe query
    def query_lowd_bins(self,qidx,search_radius=1,order=True):
        if not hasattr(self,'lowd_bins'):
            raise ValueError('low dimensional bins for model not created')
        query_bin=self.lowd_bins[self.lowd_pointstobins[qidx]]
        valid_bins=np.flatnonzero((query_bin[None,:]^self.lowd_bins).sum(axis=1)<=search_radius)
        #valid_bins=np.flatnonzero((query_bin[None,:]^self.lowd_bins).sum(axis=1)<=2*search_radius)
        all_points=reduce(np.union1d,np.array([self.lowd_binstopoints[idx] for idx in valid_bins]))
        if order:
            l1distances=(self.hashes[qidx,:]^self.hashes[all_points,:]).sum(axis=1)
            all_points=all_points[l1distances.argsort()]
        all_points=all_points[1:self.nnn+1]
        return all_points
    
    #unused
    def query_highd_bins(self,qidx,order=False):
        if not hasattr(self,'highd_bins'):
            raise ValueError('high dimensional bins for model not created')
        valid_bins=self.highd_pointstobins[qidx]
        all_points=reduce(np.union1d,np.array([self.highd_binstopoints[idx] for idx in valid_bins]))
        if order:
            l1distances=(self.hashes[qidx,:]^self.hashes[all_points,:]).sum(axis=1)
            all_points=all_points[l1distances.argsort()]
        return all_points
    
    #get confusion matrix your choose the radius to replace 40 or just use the query function without multiprobe
    def get_confusion_matrix(self,indices):
        cm = np.zeros((self.group_counts,self.group_counts))
        for idx in indices:
            # multiprobe version
            #pre_label = self.label[self.query_lowd_bins(idx,40)]
            #if pre_label.shape[0] ==0:
            #    pre_label = self.label[self.query_lowd_bins(idx,40*2)]
            
            # no-multiprobe version
            pre_label = self.label[self.query(idx,self.nnn)]
            
            true_label = np.array([self.label[idx]]*pre_label.shape[0])
            cm += confusion_matrix(true_label,pre_label,np.unique(self.label))
        return cm
    
    def test_performance(self):
        if not hasattr(self,'bins'):
            self.create_lowd_bins()
        CKS = 0
        for step in range(5):
            query_indices = np.random.choice(self.data.shape[0],self.data.shape[0]//5)
            CM = self.get_confusion_matrix(query_indices)
            CKS += self.compute_CKS(CM)/5
        return CKS  

# DenseFly implementation

In [ ]:
class denseflylsh(flylsh):
    def __init__(self,data,label,hash_length,nnn,sampling_ratio,embedding_size):
        """
        data: Nxd matrix
        label: N*1 vector
        hash_length: scalar
        nnn: n nearest neighbors
        sampling_ratio: fraction of input dims to sample from when producing a hash
        embedding_size: dimensionality of projection space, m
        Note that in Flylsh, the hash length and embedding_size are NOT the same
        whereas in usual LSH they are
        """
        self.hash_length=hash_length
        self.embedding_size=embedding_size
        K=embedding_size//hash_length
        self.data=data-np.mean(data,axis=1,keepdims=True)
        self.label=label
        self.group_counts=np.unique(self.label).shape[0]
        self.nnn=nnn
        
        weights=np.random.random((data.shape[1],embedding_size))
        self.weights=(weights>1-sampling_ratio) #sparse projection vectors
        all_activations=(self.data@self.weights)
        threshold=0
        self.hashes=(all_activations>=threshold) #choose topk activations
        self.dense_activations=all_activations
        self.sparse_activations=self.hashes.astype(np.float32)*all_activations #elementwise product
        self.maxl1distance=2*self.hash_length
        self.lowd_hashes=all_activations.reshape((-1,hash_length,K)).sum(axis=-1) > 0
        
        self.create_lowd_bins()

# Experiment 1: Self-mapping

mapping cells within one datset: use one cell as a query and use remaining cells as the reference. 

Please run `Rscript data_generator\SIM_I.R` to obtain the simulation dataset.

In [ ]:
# get data and label
path = "SelfMapping_Data.txt"    
data = pd.read_table(path,sep=' ')
labels_=np.array(data.iloc[:,-1])
inputs_=np.array(data.iloc[:,:-1])

for hash_length in [64,128,256,512,1024]:
    print('hash length:',hash_length)
    # set parameters
    nnn=1
    # construct models
    lshmodel = LSH(inputs_,labels_,hash_length,nnn)
    flymodel = flylsh(inputs_,labels_,hash_length,nnn,0.1,20*hash_length)
    denseflymodel = denseflylsh(inputs_,labels_,hash_length,nnn,0.1,20*hash_length)
    # test performance
    print(lshmodel.test_performance())
    print(flymodel.test_performance())
    print(denseflymodel.test_performance())

# Experiment 2: Cross-batch mapping

mapping cells of the same type across two batches: use cells from one batch  and use cells of the other batch as the reference. 

Please run `Rscript data_generator\SIM_II.R` to obtain the simulation dataset.

In [ ]:
def test_batcheffect(model_1,model_2):
    # query cells from batch 1 with batch 2 data as the reference
    cm = 0
    for i in range(5):
        query_indices = np.random.choice(model_1.data.shape[0],model_1.data.shape[0]//5)
        for index in query_indices:
            NNs = model_2.query(model_1.data[index],model_2.nnn)
            pre_label= model_2.label[NNs]
            true_label = np.array([model_1.label[index]]*pre_label.shape[0])
            cm += confusion_matrix(true_label,pre_label,np.unique(model_2.label))
    return cm

In [ ]:
# read data and divide matrix into two batches and their labels
path = "Batch_Data.txt"    
data=pd.read_table(path,sep=' ')
index_b1=data["Batch"]=="Batch1"
index_b2=data["Batch"]=="Batch2"
data_1 = np.array(data[index_b1].iloc[:,:-2])
label_1 = np.array(data[index_b1].iloc[:,-1])
data_2 = np.array(data[index_b2].iloc[:,:-2])
label_2 = np.array(data[index_b2].iloc[:,-1])

In [ ]:
#setting parameters
hash_length=64 # hash length can be selected from [64,128,256,512,1024]
nnn=10
#construct DenseFly models
print('DenseFly')
densemodel_1=denseflylsh(data_1,label_1,hash_length,nnn,0.1,20*hash_length)
densemodel_2=denseflylsh(data_2,label_2,hash_length,nnn,0.1,20*hash_length)
#get confusion_matrix and compute cohen kappa score
cm_1to2 = test_batcheffect(densemodel_1,densemodel_2)
cm_2to1 = test_batcheffect(densemodel_2,densemodel_1)
compute_CKS(cm_1to2),compute_CKS(cm_2to1)

#construct FlyHash models
print('FlyHash')
flymodel_1=flylsh(data_1,label_1,hash_length,nnn,0.1,20*hash_length)
flymodel_2=flylsh(data_2,label_2,hash_length,nnn,0.1,20*hash_length)
#get confusion_matrix and compute cohen kappa score
cm_1to2 = test_batcheffect(flymodel_1,flymodel_2)
cm_2to1 = test_batcheffect(flymodel_2,flymodel_1)
compute_CKS(cm_1to2),compute_CKS(cm_2to1)

#construct SimHash models
print('SimHash')
lshmodel_1=LSH(data_1,label_1,hash_length,nnn)
lshmodel_2=LSH(data_2,label_2,hash_length,nnn)
#get confusion_matrix and compute cohen kappa score
cm_1to2 = test_batcheffect(lshmodel_1,lshmodel_2)
cm_2to1 = test_batcheffect(lshmodel_2,lshmodel_1)
compute_CKS(cm_1to2),compute_CKS(cm_2to1)

# Experiment 3: Dropout events

mapping cells within one datset: Try different dropout rate. 

Please run `Rscript data_generator\SIM_III.R` to obtain the simulation dataset.

In [ ]:
# test robust of dropout

# get data and label
path = "Dropout_X_Data.txt"  #!!!!####### please change X to 0,1,2,3,4,5 #########!!!!#
data = pd.read_table(path,sep=' ')
labels_=np.array(data.iloc[:,-1])
inputs_=np.array(data.iloc[:,:-1])

for i in [32,64,128,256]:
    print(i)
    # set parameters
    hash_length=i
    nnn=1
    # construct models
    lshmodel = LSH(inputs_,labels_,hash_length,nnn)
    flymodel = flylsh(inputs_,labels_,hash_length,nnn,0.1,20*hash_length)
    denseflymodel = denseflylsh(inputs_,labels_,hash_length,nnn,0.1,20*hash_length)
    # test performance
    print(lshmodel.test_performance())
    print(flymodel.test_performance())
    print(denseflymodel.test_performance())

# Time consumption

In [ ]:
# test the relationship between cell numbers and query time

# get data and label
path = "SelfMapping_Data.txt" 
data = pd.read_table(path,sep=' ')
labels_=np.array(data.iloc[:,-1])
inputs_=np.array(data.iloc[:,:-1])

#chooses subset
indexs=np.random.choice(range(2000),xxx)   ### please chaneg xxx to the sample number you want , such as 200
inputs=inputs_[indexs,:]
labels=labels_[indexs]

#set parameters
hash_length=128
nnn=1
radius=40

# construct models
lshmodel = LSH(inputs,labels,hash_length,nnn)
flymodel = flylsh(inputs,labels,hash_length,nnn,0.1,20*hash_length)
denseflymodel = denseflylsh(inputs,labels,hash_length,nnn,0.1,20*hash_length)

#test time
start = time.time()
for i in range(100):
    lshmodel.query_bins(i,nnn,radius)
print('SimHash:',time.time()-start)

start = time.time()
for i in range(100):
    flymodel.query_lowd_bins(i,nnn,radius)
print('FlyHash:',time.time()-start)

start = time.time()
for i in range(100):
    denseflymodel.query_lowd_bins(i,nnn,radius)
print('DenseFly:',time.time()-start)